In [3]:
import sqlite3, os, sys
import pandas as pd
import tensorflow as tf
import numpy as np
from tqdm.notebook import tqdm

In [1]:
db_path = "../db_files/dev_level7_mu_e_tau_oscweight_000/dev_level7_mu_e_tau_oscweight_000.db"

In [4]:
conn = sqlite3.connect(db_path)

In [5]:
cursor = conn.cursor()

In [5]:
cursor.execute(f"SELECT name FROM sqlite_master WHERE type='table';")

In [6]:
print(cursor.fetchall())

[]


In [8]:
df_feat=pd.read_sql("select * from features limit 100", conn)

,event_no,charge_log10,dom_time,dom_x,dom_y,dom_z
0,111145172,-0.583333,2.405660,0.000000,0.255849,-0.936072
1,111145172,0.583333,5.239623,-2.468251,1.038192,-1.074582
2,111145172,0.083333,0.096226,-0.965874,1.228854,-0.290742
3,111145172,-0.666667,1.420755,-0.965874,1.228854,-0.290742
4,111145172,0.083333,-0.371698,-0.965874,1.228854,-0.428203
...,...,...,...,...,...,...
95,111145175,0.916667,0.666038,0.317855,-0.061288,0.033740
96,111145175,0.583333,1.926415,0.317855,-0.061288,-0.192590
97,111145175,-0.916667,1.984906,0.317855,-0.061288,-0.192590
98,111145175,0.500000,0.660377,-0.057160,0.959408,0.728146


In [11]:
df_truth=pd.read_sql("select * from truth limit 1000000", conn)

In [13]:
n_data = 100
start_id = conn.execute(f"select distinct event_no from truth where pid = 13")
stop_id  = conn.execute(f"select distinct event_no from truth where pid = 13 limit 1 offset {n_data}")

In [15]:
start_id.fetchall()

[]

In [12]:
stop_id.fetchall()

[(138674472,)]

In [16]:
start, stop = 0, 100000
# start_id  = conn.execute(f"select distinct event_no from features where pid = 13 limit ")
event_nos = pd.read_sql(f"select distinct event_no from features limit {stop} offset {start}", conn)

In [17]:
event_nos

,event_no
0,120102888
1,120102889
2,120102890
3,120102891
4,120102892
...,...
99995,120228027
99996,120228028
99997,120228029
99998,120228030


In [23]:
start, stop = 0, 100000
select_rows = ["dom_x", "dom_y", "dom_z", "time", "charge_log10"]

start_id = conn.execute(f"select distinct event_no from features limit 1 offset {start}").fetchall()[0][0]
stop_id  = conn.execute(f"select distinct event_no from features limit 1 offset {stop}").fetchall()[0][0]

In [24]:
features = ["dom_x", "dom_y", "dom_z", "time", "charge_log10"]
targets  = ["energy_log10", "position_x", "position_y", "position_z", "direction_x", "direction_y", "direction_z"]
feature_call = ", ".join(features)
target_call  = ", ".join(targets)
df_event = pd.read_sql(f"select event_no       from features where event_no >= {start_id} and event_no < {stop_id}", conn)
df_feat  = pd.read_sql(f"select {feature_call} from features where event_no >= {start_id} and event_no < {stop_id}", conn)
df_targ  = pd.read_sql(f"select {target_call } from truth    where event_no >= {start_id} and event_no < {stop_id}", conn)

In [25]:
idx_list    = np.array(df_event)
x_not_split = np.array(df_feat)

In [30]:
np.shape(idx_list)

(5965924, 1)

In [31]:
ids, idx, counts = np.unique(idx_list.flatten(), return_index = True, return_counts = True) 

In [29]:
np.sum(counts)

5965924

In [34]:
x_not_split.shape

(5965924, 5)

In [35]:
xs = np.split(x_not_split, np.cumsum(counts))

In [61]:
ys = np.array(df_targ)

In [43]:
np.shape(xs)

C:\Users\chris\anaconda3\lib\site-packages\numpy\core\_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


(100001,)

In [24]:
for x, y in zip(xs, ys):
    print(x, y)
    break

[[-1.84428951 -1.22482404  0.39792449 -1.56428307  0.16261479]
 [-0.2078301  -0.18691354 -0.44418075 -1.68247499 -1.92155384]
 [ 0.06322446 -0.29591152 -0.46829432 -0.05335309 -0.38592265]
 [ 0.06322446 -0.29591152 -0.14411839  0.90848462  0.23806267]
 [ 0.06322446 -0.29591152 -0.14411839 -1.41830308  1.49374495]
 [ 0.06322446 -0.29591152 -0.08772599 -0.02074843 -1.65692291]
 [ 0.06322446 -0.29591152 -0.08772599 -0.03223416  0.44497857]
 [ 0.          0.29478052 -0.12298133  0.44979622 -0.38592265]
 [-0.04194698 -0.15939176 -0.25229772 -0.30048166  0.        ]
 [-0.04194698 -0.15939176 -0.23821973 -0.30085217  0.89259397]
 [-0.04194698 -0.15939176 -0.2241216  -0.29529455 -1.0557719 ]
 [-0.04194698 -0.15939176 -0.1818273  -0.26083735 -0.62103541]
 [-0.07899002  2.28041226  1.09275381  1.07373101  0.50835111]
 [ 0.74159036  1.95723031  0.97476017  1.00629863  0.68454405]
 [-0.25763961  1.79339814  0.32990768  0.42645424  0.31007972]
 [-0.25763961  1.79339814  0.32990768 -1.44942571  0.08

In [63]:
from sklearn.neighbors import kneighbors_graph as knn
from spektral.data import Graph
neighbors=6

In [65]:
print("Generating adjacency matrices")
graph_list = []
for x, y in tqdm(zip(xs, ys), total = len(xs)):
    a = knn(x[:, :3], neighbors)



    graph_list.append(Graph(x = x, a = a, y = y))

graph_list = np.array(graph_list, dtype = object)

Generating adjacency matrices


In [51]:
s = [x.shape[0] for x in xs]

In [ ]:
xs = np.array(xs, dtype = object)[np.argsort(s)]

In [57]:
size, idx = np.unique(np.sort(s), return_index = True)

In [ ]:
preprocess(list(xs[4: 9]))

In [53]:
np.sort(s)

array([ 14,  19,  19, ..., 822, 876, 913])

In [46]:
idx

array([    0,     1,     4,     9,    21,    45,    85,   143,   244,
         401,   652,  1016,  1483,  2097,  2924,  4001,  5282,  6731,
        8466, 10599, 13093, 15805, 18756, 21801, 24937, 28170, 31567,
       34857, 38137, 41355, 44535, 47573, 50422, 53228, 55770, 58086,
       60383, 62484, 64403, 66274, 67917, 69509, 70992, 72361, 73664,
       74887, 75978, 76999, 77932, 78771, 79617, 80384, 81125, 81818,
       82484, 83089, 83658, 84237, 84756, 85227, 85712, 86149, 86570,
       86957, 87318, 87719, 88092, 88425, 88746, 89059, 89334, 89610,
       89868, 90105, 90354, 90606, 90834, 91059, 91268, 91484, 91711,
       91916, 92108, 92283, 92486, 92656, 92805, 92975, 93132, 93295,
       93449, 93584, 93732, 93878, 94005, 94124, 94236, 94346, 94449,
       94538, 94645, 94733, 94844, 94928, 95026, 95123, 95213, 95306,
       95396, 95483, 95584, 95668, 95763, 95835, 95906, 95966, 96052,
       96127, 96195, 96259, 96320, 96387, 96454, 96511, 96563, 96613,
       96675, 96739,